In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

In [2]:
import sys
sys.path.insert(1, "../../")

In [3]:
from train import *
from data_preprocessing import *
from Models.darknet19 import *

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
model = Darknet19(num_classes=10, device=device, dtype=torch.float32)

In [6]:
data_path = '../../../datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [7]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [8]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [9]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations, half=False, show=False)

In [10]:
epochs=3
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [12]:
loss_fn = nn.NLLLoss()

In [13]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=epochs-2, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('../log/darknet19/training'),
        record_shapes=True,
        profile_memory=True,
        with_flops=True,
        with_modules=True,
        with_stack=True)


Profiling your personal module 
https://pytorch.org/tutorials/beginner/profiler.html

In [17]:
history, gradient_stats, prof = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, prof, outputs_path = '../log/darknet19/training/')

2024-11-22 11:01:24.680987 Epoch 1 
2024-11-22 11:01:46.711506 Batch 1 
2024-11-22 11:01:47.880380 Batch 2 
2024-11-22 11:01:48.148064 Batch 3 
2024-11-22 11:01:48.438181 Batch 4 
2024-11-22 11:01:48.721934 Batch 5 
2024-11-22 11:01:49.000089 Batch 6 
2024-11-22 11:01:49.279553 Batch 7 
2024-11-22 11:01:49.564535 Batch 8 
2024-11-22 11:01:49.842579 Batch 9 
2024-11-22 11:01:50.114937 Batch 10 
2024-11-22 11:01:50.409177 Batch 11 
2024-11-22 11:01:50.681579 Batch 12 
2024-11-22 11:01:50.964618 Batch 13 
2024-11-22 11:01:51.242975 Batch 14 
2024-11-22 11:01:51.515636 Batch 15 
2024-11-22 11:01:51.818951 Batch 16 
2024-11-22 11:01:52.115204 Batch 17 
2024-11-22 11:01:52.417853 Batch 18 
2024-11-22 11:01:52.711917 Batch 19 
2024-11-22 11:01:53.012965 Batch 20 
2024-11-22 11:01:53.309751 Batch 21 
2024-11-22 11:01:53.609211 Batch 22 
2024-11-22 11:01:53.913114 Batch 23 
2024-11-22 11:01:54.209177 Batch 24 
2024-11-22 11:01:54.521532 Batch 25 
2024-11-22 11:01:54.823337 Batch 26 
2024-11-22 

In [31]:
print(prof.key_averages().table(row_limit=-1))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.63%        1.383s       100.00%      218.369s       24.263s      73.62 Mb      -1.99 Gb             9            --  
                                          Training Loop         0.02%      50.267ms        51.74%      112.981s       12.553s      73.60 Mb    -155.05 Mb             9            --  
                     Validation Loop Before Data Loader         4.08%        8.9

In [32]:
save_train_outputs('../log/darknet19/training', history, gradient_stats)